# Create a slab with specified termination

Create a slab with a specified termination from a bulk material. 

<h2 style="color:green">Usage</h2>


1. Make sure to select Input Materials (in the outer runtime) before running the notebook.
1. Set notebook parameters in cell 1.1. below (or use the default values).
1. Click “Run” > “Run All” to run all cells. 
1. if prompted, select the termination for the slab in cell 2.5.
1. Scroll down to view results. 


## Notes

1. For more information, see [Introduction](Introduction.ipynb)
<!-- # TODO: use a hashtag-based anchor link to interface creation documention above -->


## 1. Prepare the Environment
### 1.1. Set up defect parameters 

In [27]:
# Enable interactive selection of terminations via UI prompt
IS_TERMINATIONS_SELECTION_INTERACTIVE = False 

MILLER_INDICES = (0, 1, 1)
THICKNESS = 3  # in atomic layers
VACUUM = 10.0  # in angstroms
XY_SUPERCELL_MATRIX = [[1, 0], [0, 1]]
USE_ORTHOGONAL_Z = True
USE_CONVENTIONAL_CELL = True

# Index of the termination to be selected
TERMINATION_INDEX = 0

### 1.2. Install Packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install` (see [README](../../README.ipynb)).

In [28]:
import sys

if sys.platform == "emscripten":
    import micropip
  
    await micropip.install('mat3ra-api-examples', deps=False)
    from utils.jupyterlite import install_packages
    await install_packages("")

### 1.3. Get input materials

In [57]:
from utils.jupyterlite import get_materials
from mat3ra.made.tools.modify import rotate

materials = get_materials(globals())
material = materials[31]
original_material = material
material = rotate(material, axis=[1,0,0], angle=10, rotate_cell=False)

0: 0-Ni
1: 1-Graphene
2: 10-Al2O3, Sapphire, RHL (R-3c) 3D (Bulk), mp-1143 (1)
3: 11-ZnO, Zinc Oxide, HEX (P6_3mc) 3D (Bulk), mp-2133
4: 12-Cd4 Te4
5: 13-Si4 C4
6: 14-GaN, Gallium Nitride, HEX (P6_3mc) 3D (Bulk), mp-804
7: 15-WS2, Tungsten Disulfide, HEX (P-6m2) 2D (Monolayer), 2dm-3749
8: 16 - In4 P4
9: 17 Graphene (Zigzag nanoribbon)
10: 4-Te2Mo
11: 5-HfO2
12: 6-Ni4(110), termination Ni_Pmmm_2, Slab, Terrace, 1 steps, [2 0 0]
13: 7-Ag4
14: 8-Si, Silicene, HEX (P-3m1) 2D (Monolayer), 2dm-5934
15: 9-GaAs, Gallium Arsenide, FCC (F-43m) 3D (Bulk), mp-2534
16: 99 test material
17: C, Graphene, HEX (P6:mmm) 2D (Monolayer), 2dm-3993 (Zigzag nanoribbon)
18: Cu4(111)-Cu4(001), Grain Boundary
19: Custom Shape - Etching Pattern
20: Custom Shape - Transistor
21: Graphene (Zigzag nanoribbon)
22: Ni, Nickel, FCC (Fm-3m) 3D (Bulk), mp-23 passivated
23: Ni25 C1, Substitution C Defect, Pair  Defect, Vacancy and Substitution Defect Pair
24: Ni4(001), termination Ni_P4:mmm_2, Slab with box cutout
25: O

### 1.4. Preview the material

In [58]:
from utils.visualize import visualize_materials as visualize

visualize([original_material, material], repetitions=[3, 3, 1], rotation="0x")
visualize([original_material, material], repetitions=[3, 3, 1], rotation="-90x")

GridBox(children=(VBox(children=(Label(value='O3SrTi - Material - rotation: 0x', layout=Layout(align_self='cen…

GridBox(children=(VBox(children=(Label(value='O3SrTi - Material - rotation: -90x', layout=Layout(align_self='c…

## 2. Configure slab and select termination

### 2.1. Create slab configuration
Slab Configuration lets define the slab thickness, vacuum, and the Miller indices of the interfacial plane and get the slabs with possible terminations.


In [59]:

from mat3ra.made.tools.build.slab import SlabConfiguration

slab_configuration = SlabConfiguration(
    bulk=material,
    miller_indices=MILLER_INDICES,
    thickness=THICKNESS, # in atomic layers
    vacuum=VACUUM, # in angstroms
    xy_supercell_matrix=XY_SUPERCELL_MATRIX,
    use_orthogonal_z=USE_ORTHOGONAL_Z,
    use_conventional_cell=USE_CONVENTIONAL_CELL,
)

### 2.2. Get possible terminations for the slabs

In [60]:
from mat3ra.made.tools.build.slab import get_terminations
slab_terminations = get_terminations(slab_configuration)

### 2.3. Visualize slabs for all possible terminations

In [61]:
from mat3ra.made.tools.build.slab import create_slab

slabs = [create_slab(slab_configuration, termination) for termination in slab_terminations]

visualize([{"material":slab, "title": slab.metadata["build"]["termination"]} for slab in slabs ], repetitions=[3, 3, 1], rotation="-90x")

GridBox(children=(VBox(children=(Label(value='O9Sr3Ti3 - TiO_Pmmm_2 - rotation: -90x', layout=Layout(align_sel…


### 2.4. Print terminations for the Slab

In [62]:
print("Terminations")
for idx, termination in enumerate(slab_terminations):
    print(f"    {idx}: {termination}")

Terminations
    0: TiO_Pmmm_2
    1: O2_Pmmm_2
    2: Sr_Pmmm_1


### 2.5. Select termination for the Slab

In [ ]:
from utils.io import ui_prompt_select_array_element_by_index, ui_prompt_select_array_element_by_index_pyodide

termination_index = TERMINATION_INDEX

termination = slab_terminations[termination_index]
if IS_TERMINATIONS_SELECTION_INTERACTIVE:
    if sys.platform == "emscripten":
        termination = await ui_prompt_select_array_element_by_index_pyodide(slab_terminations, element_name="termination")
    else:
        termination = ui_prompt_select_array_element_by_index(slab_terminations, element_name="termination")

## 3. Create the slab with selected termination

In [ ]:
from mat3ra.made.tools.build.slab import create_slab

slab = create_slab(slab_configuration, termination)

## 4. Visualize the slab

In [ ]:
visualize(slab, repetitions=[1, 1, 1], rotation="0x")

# 5. Pass material to the outside runtime

In [ ]:
from utils.jupyterlite import set_materials
set_materials(slab)